### Jupyter Notebook to compute morphological metrics from axon and myelin segmentation of a microscopy sample.

- To use it, you need the paths of the microscopy image and the segmentation prediction (output of AxonDeepSeg).


#### Imports

In [ ]:
import numpy as np
import math
import os
import matplotlib.pyplot as plt

from skimage import measure, morphology, feature


from AxonDeepSeg.morphometrics.compute_morphometrics import *
import AxonDeepSeg.ads_utils as ads
from config import axonmyelin_suffix

%matplotlib inline

#### Read image and segmented prediction

In [ ]:
# Change the image and segmentation paths here for your sample:
path_img = '../AxonDeepSeg/models/default_SEM_model/data_test/image.png'
path_pred = '../AxonDeepSeg/models/default_SEM_model/data_test/image' + str(axonmyelin_suffix)

In [ ]:
# Read paths and compute axon/myelin masks
img = ads.imread(path_img)
pred = ads.imread(path_pred)
pred_axon = pred > 200
pred_myelin = np.logical_and(pred >= 50, pred <= 200)
path_folder, file_name = os.path.split(path_img)

### Step-by-step use case: extract morphometrics from a segmented microscopy sample

#### 1) Get axon morphometrics

In [ ]:
# By default for computing the morphometrics, AxonDeepSeg treats the axon shape as circle.  
# If you wish to consider shape of axon as an ellipse set the axon_shape variable to 'ellipse'.

axon_shape = 'circle'   '''String, circle: axon shape is considered circle for computing morphometrics.
                                           Equivalent diameter of axon is used to compute the axon_diam,                                                myelin_thickness, axonmyelin_diam, gratio.
                                        
                                   ellipse: axon shape is considered ellipse for computing morphometrics.                                                Ellipse Minor axis is used to compute the axon_diam, myelin                                                  thickness, axonmyelin_diam and gratio.
                        '''       

# Computes axon morphometrics
stats_array = get_axon_morphometrics(pred_axon,path_folder, axon_shape=axon_shape)


#### 2) Save axon morphometrics and load back

In [ ]:
save_axon_morphometrics(path_folder,stats_array)
stats_array = load_axon_morphometrics(path_folder)

#### 3) Extract and plot axon diameter distribution

In [ ]:
# Get axon diameter distribution
axon_diam_list = [d['axon_diam'] for d in stats_array]

In [ ]:
# Plot histogram of axon diameter distribution 
plt.figure(figsize=(8,6))
n, bins, patches = plt.hist(axon_diam_list,30, facecolor='g', alpha=0.7)
plt.xlabel('Axon diameter in um',fontsize=10)
plt.ylabel('Frequency',fontsize=10)
plt.title('Axon diameter distribution',fontsize=11)
plt.grid(True)

#### 4) Display boxplots of other axonal morphometrics (example with solidity and eccentricity)

In [ ]:
# Get axon solidity distribution (measure of compactness of the axon)
axon_solidity_list = [d['solidity'] for d in stats_array]
# Get axon eccentricity distribution (measure of ellipticity/extent of the axon)
axon_eccentricity_list = [d['eccentricity'] for d in stats_array]

In [ ]:
# Plot boxplots of solidity and eccentricity
plt.figure(figsize=(6,6))
data_to_plot = [axon_solidity_list,axon_eccentricity_list]
plt.boxplot(data_to_plot,patch_artist=True)
plt.title('Solidity and eccentricity distributions of axons',fontsize=11)
plt.grid(True)
frame1 = plt.gca()
frame1.set_xticklabels(['Solidity', 'Eccentricity'],fontsize=11)

#### 5) Display and save axon mask overlay labelled for axon diameter

In [ ]:
fig = draw_axon_diameter(img,path_pred,pred_axon,pred_myelin, axon_shape=shape)
from IPython.core.display import display
fig.canvas.draw()
display(fig)

#### 6) Compute aggregate axon/myelin morphometrics

In [ ]:
# Compute aggregate metrics
aggregate_metrics = get_aggregate_morphometrics(pred_axon,pred_myelin,path_folder, shape=shape)
print(aggregate_metrics)

#### 7) Save aggregate metrics

In [ ]:
write_aggregate_morphometrics(path_folder,aggregate_metrics)

#### Other useful code snippets

In [ ]:
# Show metrics available for axons
print(stats_array[0].keys())

In [ ]:
# Get axon diameter value of axon object 0
print(stats_array[0]['axon_diam'])

In [ ]:
# Get solidity value of axon object 50
print(stats_array[49]['solidity'])

In [ ]:
# Display all stats for axon object 10
print(stats_array[9])

In [ ]:
# Get axon diameter value of all axons in list
axon_diam_list = [d['axon_diam'] for d in stats_array]

In [ ]:
# Get number of axons in sample
print(np.size(axon_diam_list))

In [ ]:
# Get mean diameter
mean_diam = np.mean(axon_diam_list)
print(mean_diam)

In [ ]:
# Get max diameter
max_diam = np.max(axon_diam_list)
print(max_diam)